In [22]:

import sys, os

sys.path.append(os.path.abspath(os.path.join('../scripts')))


In [23]:
import importlib
import fetch
importlib.reload(fetch)
from fetch import FetchData

In [24]:
from shapely.geometry import shape, GeometryCollection, Polygon,Point

In [25]:
import pyproj

In [26]:
MINX, MINY, MAXX, MAXY = [-93.756155, 41.918015, -93.747334, 41.921429]
polygon = Polygon(((MINX, MINY), (MINX, MAXY), (MAXX, MAXY), (MAXX, MINY), (MINX, MINY)))

data=FetchData("IA_FullState").fetch_elevation(polygon,"EPSG:4326")

In [27]:
data[0].describe()

,elevation
count,189060.000000
mean,316.202441
std,2.537666
min,306.290000
25%,315.080000
50%,317.150000
75%,317.940000
max,319.760000


In [ ]:
data[0].head()

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
fig = plt.figure()
ax = fig.add_subplot(111,projection='3d')
ax.scatter( data[0].geometry.x,
           data[0].geometry.y,data[0].elevation, cmap='Spectral_r',s=0.001, c=data[0].elevation)

plt.show()

In [ ]:
# import geoplot
# import geoplot.crs as gcrs
# geoplot.kdeplot(data[0],shade=True, cmap='Spectral_r',projection=gcrs.WebMercator(), shade_lowest=True)

In [ ]:
# import numpy as np
# x=data[0].geometry.x
# y=data[0].geometry.y
# z = np.array(data[0].elevation)

# z = z.reshape((len(x), len(y)))
# plt.contour(x,y,z, colors='black');

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 1)


data[0].plot(column='elevation',  cmap = 'Spectral_r', ax=ax, legend=True)
plt.show()

In [28]:
import numpy as np
from scipy.interpolate import griddata

data_meters= data[0].to_crs({'init': 'epsg:3395'})

points = [[i.x,i.y] for i in data_meters.geometry.values]
values = data_meters['elevation'].values

D:\Users\same\anaconda3\envs\pdalworkshop\lib\site-packages\pyproj\crs\crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [ ]:
from math import floor

In [ ]:
xDim = floor(data_meters.total_bounds[2])-floor(data_meters.total_bounds[0])
yDim = floor(data_meters.total_bounds[3])-floor(data_meters.total_bounds[1])
rasterRes = 2
nCols = xDim / rasterRes
nRows = yDim / rasterRes

print('Raster X Dim: %.2f, Raster Y Dim: %.2f'%(xDim,yDim))
print('Number of cols:  %.2f, Number of rows: %.2f'%(nCols,nRows)) #Check if the cols and row don't have decimals

In [ ]:
grid_y, grid_x = np.mgrid[floor(data_meters.total_bounds[1])+rasterRes/2:floor(data_meters.total_bounds[3])-rasterRes/2:nRows*1j,
                         floor(data_meters.total_bounds[0])+rasterRes/2:floor(data_meters.total_bounds[2])+rasterRes/2:nCols*1j]

In [ ]:
mtop = griddata(points, values, (grid_x, grid_y), method='cubic')
# mtop[:10,:10]

In [ ]:
values.max()

In [ ]:

%matplotlib widget

import matplotlib.pyplot as plt

plt.imshow(mtop, cmap="Spectral_r")
plt.colorbar()
plt.show()

In [ ]:
frame.elevation=frame.elevation.apply(fl)

In [34]:
%matplotlib widget
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
fig = plt.figure()
ax = fig.add_subplot(111,projection='3d')
ax.scatter( frame.geometry.x,
           frame.geometry.y,frame.elevation, cmap='Spectral_r',s=0.001, c=frame.elevation)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
type(mtop[0][1])

In [ ]:
# from scipy import interpolate
# x= data_meters.geometry.x
# y=data_meters.geometry.y
# z=data_meters.elevation.values
# f = interpolate.interp2d(x, y, z, kind='linear')

In [29]:
totalPointsArray = np.zeros([data_meters.shape[0],3])
#iteration over the geopandas dataframe
for index, point in data_meters.iterrows():
    pointArray = np.array([point.geometry.coords.xy[0][0],point.geometry.coords.xy[1][0],point['elevation']])
    totalPointsArray[index] = pointArray
totalPointsArray[:5,:]

array([[-1.04368762e+07,  5.12015680e+06,  3.10100000e+02],
       [-1.04368319e+07,  5.12017915e+06,  3.10560000e+02],
       [-1.04368762e+07,  5.12024565e+06,  3.10400000e+02],
       [-1.04368765e+07,  5.12031223e+06,  3.11590000e+02],
       [-1.04368549e+07,  5.12033398e+06,  3.12160000e+02]])

In [30]:

from matplotlib.tri import Triangulation, LinearTriInterpolator,CubicTriInterpolator
#triangulation function
triFn = Triangulation(totalPointsArray[:,0],totalPointsArray[:,1])
#linear triangule interpolator funtion
linTriFn = LinearTriInterpolator(triFn,totalPointsArray[:,2])

In [ ]:
from shapely.geometry import Point

In [31]:
rasterRes = 1

xCoords = np.arange(totalPointsArray[:,0].min(), totalPointsArray[:,0].max()+rasterRes, rasterRes)
yCoords = np.arange(totalPointsArray[:,1].min(), totalPointsArray[:,1].max()+rasterRes, rasterRes)
zCoords = np.zeros([yCoords.shape[0],xCoords.shape[0]])
polygons=[]
elevations=[]

#loop among each cell in the raster extension
for indexX, x in np.ndenumerate(xCoords):
    for indexY, y in np.ndenumerate(yCoords):
        tempZ = linTriFn(x,y)
        #filtering masked values
        if tempZ == tempZ:
            zCoords[indexY,indexX]=tempZ
            
            polygons.append(Point(x,y)) 
            elevations.append(float(tempZ.data[()]))
        else:
            zCoords[indexY,indexX]=np.nan
            # elevations.append(np.nan)


In [32]:
import geopandas as gpd

In [33]:
frame=gpd.GeoDataFrame(columns=["elevation", "geometry"])
frame['elevation'] = elevations
frame['geometry'] = polygons
frame.set_geometry("geometry",inplace=True)
frame.set_crs(crs=data_meters.crs, inplace=True)

,elevation,geometry
0,309.942788,POINT (-10436886.430 5120154.043)
1,310.012600,POINT (-10436886.430 5120155.043)
2,310.047287,POINT (-10436886.430 5120156.043)
3,310.033208,POINT (-10436886.430 5120157.043)
4,310.032842,POINT (-10436886.430 5120158.043)
...,...,...
499250,318.257498,POINT (-10435906.430 5120656.043)
499251,318.354460,POINT (-10435906.430 5120657.043)
499252,318.352785,POINT (-10435906.430 5120658.043)
499253,318.333323,POINT (-10435906.430 5120659.043)


In [ ]:
frame.plot(column='elevation',  cmap = 'Spectral_r',  legend=True)
# plt.grid()

In [37]:
frame['lon']= frame['geometry'].x
frame['lat'] = frame['geometry'].y

In [ ]:
plt.imshow(zCoords,origin='lower',cmap="Spectral_r" )